In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames: 
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/final-df/final_df.csv


In [4]:
final_df =pd.read_csv('/kaggle/input/final-df/final_df.csv')

In [5]:
X = final_df.drop(columns=['runs_x'])
y = final_df['runs_x'] 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [6]:
X_train


,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five_overs_runs
35133,South Africa,Australia,Melbourne,98,4,3,5.068966,31.0
13949,India,South Africa,Durban,69,73,8,8.808511,47.0
35079,Sri Lanka,India,Mumbai,10,106,9,4.285714,10.0
7921,Sri Lanka,West Indies,Bangalore,72,38,5,5.268293,24.0
22152,Sri Lanka,New Zealand,Auckland,102,27,3,6.580645,36.0
...,...,...,...,...,...,...,...,...
50057,India,South Africa,Cape Town,31,100,9,9.300000,31.0
32511,Sri Lanka,England,Southampton,41,86,9,7.235294,39.0
5192,England,India,London,96,36,7,6.857143,29.0
12172,New Zealand,Bangladesh,Pallekele,0,116,10,0.000000,0.0


In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error 

In [8]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')  

In [9]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=1)) 
])

In [10]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test,y_pred)) 
print(mean_absolute_error(y_test,y_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.9648124954847874
2.369048341466993


In [12]:
import pickle

In [13]:
pickle.dump(pipe,open('pipe.pkl','wb'))  

In [14]:
! pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 27.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 82.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2
  Attempting uninstall: python-multipart
    Found existing installation: python-multipart 0.0.9
    Uninstalling python-multipart-0.0.9:
      Successfully uninstalled python-multipart-0.0.9
  Attempting uninstall: starlette
    Found existing installation: starlette 0.37.2
    Uninstalling starlette-0.37.2:
      Successfully uninstalled starlette-0.37.2
  Attempting uninstall: fastapi
    Found existing installation: f

In [15]:
pipe = pickle.load(open('/kaggle/working/pipe.pkl', 'rb'))
test_input = pd.DataFrame({
    'batting_team': ['India'],
    'bowling_team': ['Australia'],
    'city': ['Delhi'],
    'current_score': [120],
    'balls_left': [30],
    'wickets_left': [7],
    'crr': [8.0],
    'last_five_overs_runs': [40]
})
print(pipe.predict(test_input)) 

[175.55087]


In [16]:
import pickle
import pandas as pd
import gradio as gr
import sklearn 
# Load the pre-trained pipeline
pipe = pickle.load(open('/kaggle/working/pipe.pkl', 'rb'))

# List of teams and cities
teams = ['Australia', 'India', 'Bangladesh', 'New Zealand', 'South Africa',
         'England', 'West Indies', 'Afghanistan', 'Pakistan', 'Sri Lanka']

cities = ['Colombo', 'Mirpur', 'Johannesburg', 'Dubai', 'Auckland', 'Cape Town',
          'London', 'Pallekele', 'Barbados', 'Sydney', 'Melbourne', 'Durban',
          'St Lucia', 'Wellington', 'Lauderhill', 'Hamilton', 'Centurion',
          'Manchester', 'Abu Dhabi', 'Mumbai', 'Nottingham', 'Southampton',
          'Mount Maunganui', 'Chittagong', 'Kolkata', 'Lahore', 'Delhi',
          'Nagpur', 'Chandigarh', 'Adelaide', 'Bangalore', 'St Kitts',
          'Cardiff', 'Christchurch', 'Trinidad']

# Prediction function
def predict_score(batting_team, bowling_team, city, current_score, overs, wickets, last_five_overs_runs):
    # Compute derived features
    balls_left = 120 - (overs * 6)
    wickets_left = 10 - wickets
    crr = current_score / overs

    # Create input DataFrame
    input_df = pd.DataFrame(
        {
            'batting_team': [batting_team],
            'bowling_team': [bowling_team],
            'city': [city],
            'current_score': [current_score],
            'balls_left': [balls_left],
            'wickets_left': [wickets_left],
            'crr': [crr],
            'last_five_overs_runs': [last_five_overs_runs]
        }
    )
    # Predict score
    result = pipe.predict(input_df)
    return f"Predicted Score: {int(result[0])}"

# Gradio Interface
interface = gr.Interface(
    fn=predict_score,
    inputs=[
        gr.Dropdown(choices=sorted(teams), label="Batting Team"),
        gr.Dropdown(choices=sorted(teams), label="Bowling Team"),
        gr.Dropdown(choices=sorted(cities), label="City"),
        gr.Number(label="Current Score"),
        gr.Number(label="Overs Done (works for over > 5)"),
        gr.Number(label="Wickets Out"),
        gr.Number(label="Runs Scored in Last 5 Overs")
    ],
    outputs=gr.Textbox(label="Predicted Score"),
    title="Cricket Score Predictor",
    description="Predict the final score of a cricket team based on the match context."
)

# Launch Gradio App
interface.launch()


* Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://29671812e35247b959.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
